In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from pydataset import data

## 1. Attendance Data

Load the attendance.csv file and calculate an attendnace percentage for each student. 
One half day is worth 50% of a full day, and 10 tardies is equal to one absence.

In [2]:
attendance = pd.read_csv('untidy-data/attendance.csv')
attendance

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [3]:
attendance = attendance.rename(columns={'Unnamed: 0': 'Name'})

In [4]:
attendance = attendance.replace(['P','A','T','H'],[1,0,0.9,0.5])
attendance.set_index(keys='Name', inplace=True)
attendance.mean(axis=1)

Name
Sally    0.7625
Jane     0.6875
Billy    0.5250
John     0.9125
dtype: float64

## 2. Coffee Levels

- a. Read the coffee_levels.csv file.
- b. Transform the data so that each carafe is in it's own column.
- c. Is this the best shape for the data?

In [5]:
coffee = pd.read_csv('untidy-data/coffee_levels.csv')
coffee.head()

,hour,coffee_carafe,coffee_amount
0,8,x,0.816164
1,9,x,0.451018
2,10,x,0.843279
3,11,x,0.335533
4,12,x,0.898291


In [6]:
coffee.shape

(30, 3)

In [7]:
coffee.pivot_table(index='hour', columns='coffee_carafe')

coffee_amount                    
coffee_carafe             x         y         z
hour                                           
8                  0.816164  0.189297  0.999264
9                  0.451018  0.521502  0.915990
10                 0.843279  0.023163  0.144928
11                 0.335533  0.235529  0.311495
12                 0.898291  0.017009  0.771947
13                 0.310711  0.997464  0.398520
14                 0.507288  0.058361  0.864464
15                 0.215043  0.144644  0.436364
16                 0.183891  0.544676  0.280621
17                 0.391560  0.594126  0.436677

I believe that the pivot table is more readable than the long format.

## 3. Cake Recipes

- a. Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.
- b. Tidy the data as necessary.
- c. Which recipe, on average, is the best? recipe b
- d. Which oven temperature, on average, produces the best results? 275
- e. Which combination of recipe, rack position, and temperature gives the best result? recipe b, bottom rack, 300 degrees

In [8]:
cakes = pd.read_csv('untidy-data/cake_recipes.csv')
cakes.head()

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084


In [10]:
cakes[['recipe','position']] = cakes['recipe:position'].str.split(':', expand=True)
cakes.drop(columns={'recipe:position'})

,225,250,275,300,recipe,position
0,61.738655,53.912627,74.414730,98.786784,a,bottom
1,51.709751,52.009735,68.576858,50.228470,a,top
2,57.095320,61.904369,61.196980,99.248541,b,bottom
3,82.455004,95.224151,98.594881,58.169349,b,top
4,96.470207,52.001358,92.893227,65.473084,c,bottom
5,71.306308,82.795477,92.098049,53.960273,c,top
6,52.799753,58.670419,51.747686,56.183110,d,bottom
7,96.873178,76.101363,59.571620,50.971626,d,top


In [67]:
cakes_tidy = cakes.pivot_table(index=['recipe','position']).reset_index()

In [95]:
cakes_tidy['avg_score'] = (cakes_tidy['225'] + cakes_tidy['250'] + cakes_tidy['275'] + cakes_tidy['300']) / 4
cakes_tidy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   recipe     8 non-null      object 
 1   position   8 non-null      object 
 2   225        8 non-null      float64
 3   250        8 non-null      float64
 4   275        8 non-null      float64
 5   300        8 non-null      float64
 6   avg_score  8 non-null      float64
dtypes: float64(5), object(2)
memory usage: 576.0+ bytes


In [92]:
#recipe with highest average score

cakes_tidy.recipe[cakes_tidy['avg_score'].idxmax()]

'b'

In [125]:
# melt the tidy table so that the temperatures are in the same column
cakes_melt = cakes_tidy.melt(id_vars=['recipe', 'position'])
cakes_melt.rename(columns={'variable': 'oven_temp', 'value': 'score'}, inplace=True)

In [127]:
# group by the oven temp variable so we can get the mean scores of each oven temp
# find the index value (oven temp) with the highest average score using idxmax
cakes_melt.groupby('oven_temp').mean().idxmax()

score    275
dtype: object

In [129]:
# idxmax() returns the index that holds the max score
# loc() using the result of idxmax() and gives the observation with the highest score
    # recipe: b, position: bottom, oven: temp 300, score: 99.248541
cakes_melt.loc[cakes_melt.score.idxmax()]

recipe               b
position        bottom
oven_temp          300
score        99.248541
Name: 26, dtype: object